# training Model
## 1. setting up environment
### 1.1. Imports

In [1]:
import os
import sys

# Dynamically set the PYTHONPATH to include the project directory
project_root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
sys.path.insert(0, project_root)

import mlflow
import joblib
import torch
from torch.utils.data import DataLoader, TensorDataset
from steps.eval import *
import torch.nn as nn
from steps.logging import custom_log, get_or_create_run
from steps.loss import *
from steps.compute import *
from steps.costum_plots import *
from steps.training import *
from steps.eval import *
from steps.plot import *
from steps.model import *
import pandas as pd

### 1.2. connecting to mlflow

In [14]:
mlflow.set_tracking_uri("http://cmtcdeu82827824:5000")
experiment = mlflow.set_experiment("noise influence")

### 1.3. loading data

In [17]:
data_name = "noise/noise_u_0001"
train_path =f"../data/clean/{data_name}/train.pt"
validation_path = f"../data/clean/{data_name}/validation.pt"
test_path = f"../data/clean/{data_name}/test.pt"
metadata_path = f"../data/clean/{data_name}/metadata.csv"
scaler_path = f"../data/clean/{data_name}/scaler.pkl"

#load data
train_data = torch.load(train_path)
validation_data = torch.load(validation_path)
test_data = torch.load(test_path)
try:
    metadata = pd.read_csv(metadata_path,index_col=None)
    if metadata.empty:
        print("Die CSV-Datei ist leer. Ein leeres DataFrame wurde erstellt.")
    else:
        print("Die CSV-Datei wurde erfolgreich gelesen.")
except pd.errors.EmptyDataError:
    metadata = pd.DataFrame()
    print("Die CSV-Datei ist leer. Ein leeres DataFrame wurde erstellt.")


R = validation_data["r"][0]

scaler = joblib.load(scaler_path)
train_dataset = TensorDataset(train_data["X"],train_data["Y"])

Die CSV-Datei wurde erfolgreich gelesen.


C:\Users\JANLUKS\AppData\Local\Temp\ipykernel_6860\793037711.py:9: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

C:\Users\JANLUKS\AppData\Local\Temp\ipykernel_6860\793037711

## 2. training

In [18]:
# Define constants and initialize variables
max_epochs = 500
batchsize = 128
runs_per_size=10
parent_run_name = data_name.split("/")[1]

#parent run to encapsulate a set of runs
with mlflow.start_run(run_name=parent_run_name) as run:
    parent_run_id = run.info.run_id
    for i in range(runs_per_size):
        # train the model on the dataset
        with mlflow.start_run(run_name=f"run {i}",nested=True,parent_run_id=parent_run_id):
            mlflow.log_params(metadata.iloc[0].to_dict())
            lr = 0.001
            dataset_info= {"dataset":data_name,"random sampling":True,"num_data_points":len(train_data["X"])}
            model_parameters= {"activation":"Tanh","lr":lr,"batch_size":batchsize,"num_trained_epochs":max_epochs,"use_batchnorm":False}
            neurons_per_layer = [20,20,20]
            for i,neurons in enumerate(neurons_per_layer):
                model_parameters[f"neurons_layer_{i}"] = neurons
                
            eval_loss_fn = torch.nn.L1Loss()
            model = FluxModel(neurons_per_layer,model_parameters["use_batchnorm"])
            optimizer = torch.optim.Adam(model.parameters(),lr=lr)
            train_loader = DataLoader(train_dataset,batch_size=batchsize,shuffle=dataset_info["random sampling"])
            custom_log(dataset_info, model_parameters,train_path,validation_path,test_path,scaler_path)
            
            for epoch in range(max_epochs):
                for x,y in train_loader:
                    loss = do_training_step(model, optimizer, x,y,custom_loss)
                val_loss = validatemodel(model,validation_data,R,eval_loss_fn, epoch,loss)
                
                if epoch % 10 == 0:
                    inputs = validation_data["i_scaled"].clone().detach().requires_grad_(True)
                    pred,r = model(inputs)
                    inductances = compute_differential_inductances_fast(pred,inputs,scaler)
                    l_loss = eval_loss_fn(inductances,validation_data["l"])
                    true_state_space_parameters = torch.column_stack(physical_parameters_to_state_space(validation_data["psi"],validation_data["l"],validation_data["r"][0],validation_data["omega_el"][0]))
                    estimated_state_space_parameter= torch.column_stack(physical_parameters_to_state_space(pred,inductances,r.item(),validation_data["omega_el"][0]))
                    state_space_loss = eval_loss_fn(estimated_state_space_parameter,true_state_space_parameters)
                    mlflow.log_metrics({"val_l_mae":l_loss,"val_state_space_mae":state_space_loss}, step=epoch)
                    
            #log results
            inputs = validation_data["i_scaled"].clone().detach().requires_grad_(True)
            pred,r = model(inputs)
            inductances = compute_differential_inductances_fast(pred,inputs,scaler)
            l_loss = eval_loss_fn(inductances,validation_data["l"])
            log_l_visualization(inductances,validation_data)
            log_psi_error_plane(pred.detach(),validation_data)
            log_l_error_plane(inductances,validation_data)
            log_prediction_visualization(pred.detach(),validation_data)
            log_l_3d_subplot(inductances,validation_data)
            log_psi_3d_subplot(pred.detach(),validation_data)
            mlflow.pytorch.log_model(model,"model")
            true_state_space_parameters = torch.column_stack(physical_parameters_to_state_space(validation_data["psi"],validation_data["l"],validation_data["r"][0],validation_data["omega_el"][0]))
            estimated_state_space_parameter= torch.column_stack(physical_parameters_to_state_space(pred.detach(),inductances,r.item(),validation_data["omega_el"][0]))
            log_state_space_parameters_error_plane(true_state_space_parameters,estimated_state_space_parameter,validation_data)
            log_ss_3d_subplot(pred=estimated_state_space_parameter,true=true_state_space_parameters,validation_data=validation_data)


c:\Users\JANLUKS\Desktop\dokumente\python\ML\FluxNN\.venv\Lib\site-packages\torch\nn\modules\loss.py:128: UserWarning:

Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

2025/03/25 16:50:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run 0 at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513/runs/b414ccf5822a4217a4513c70b628e2c0
🧪 View experiment at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513


2025/03/25 16:52:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run 1 at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513/runs/eb3616cde4d94d1ab4f4710c7b418c91
🧪 View experiment at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513


2025/03/25 16:53:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run run 7 at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513/runs/de27396bc46c4a56a491273b26c360e1
🧪 View experiment at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513


2025/03/25 17:03:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run 8 at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513/runs/76f6bfe8f304413aba0bf03efa984e88
🧪 View experiment at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513


2025/03/25 17:05:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run run 9 at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513/runs/76213e1aa1c147e38a39a60c80d57f27
🧪 View experiment at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513
🏃 View run noise_u_0001 at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513/runs/d63a9453786240749137a0e73cf8f9a6
🧪 View experiment at: http://cmtcdeu82827824:5000/#/experiments/739856297498542513
